In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import area_detector_handlers.handlers

handler = area_detector_handlers.handlers.IMMHandler('data/D005_Latex67nm_dilute_att0_Lq0_001_00001-10000.imm', 1)

In [ ]:
import dask.array

images = dask.array.concatenate(handler(i) for i in range(10_000))

In [ ]:
images[0].compute()

In [ ]:
from dask.distributed import Client

client = Client()
client

In [ ]:
images

In [ ]:
from skbeam.core import correlation, roi

In [ ]:
num_levels = 7
num_bufs = 8

In [ ]:
# define the ROIs
roi_start = 30 # in pixels
roi_width = 9 # in pixels
roi_spacing = (5.0, 4.0)
x_center = 1556/2 # in pixels
y_center = 516/2 # in pixels
num_rings = 3

# get the edges of the rings
edges = roi.ring_edges(roi_start, width=roi_width, 
                       spacing=roi_spacing, num_rings=num_rings)

# get the label array from the ring shaped 3 region of interests(ROI's)
labeled_roi_array = roi.rings(
    edges, (y_center, x_center), images.shape[1:])

def overlay_rois(ax, image, label_array):
    """
    This will plot the reqiured roi's on the image
    """
    from matplotlib.colors import LogNorm
    tmp = np.array(label_array, dtype='float')
    tmp[label_array==0] = np.nan
    
    im_data = ax.imshow(image, interpolation='none', norm=LogNorm(), cmap='viridis')
    im_overlay = ax.imshow(tmp, cmap='Paired', 
                   interpolation='nearest', alpha=.5,)
    
    return im_data, im_overlay

fig, ax = plt.subplots()
plt.title("Latex 67nm")
im_data, im_overlay = overlay_rois(ax, images[0], labeled_roi_array)
plt.show()

In [ ]:
g2, lag_steps = correlation.multi_tau_auto_corr(
    num_levels, num_bufs, labeled_roi_array, images[:100])

In [ ]:
fig, ax = plt.subplots()
ax.plot(lag_steps[1:], g2[1:])